In [1]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import string
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

#1.Downloadong the text


In [43]:
f = open('alice.txt', encoding='utf8').read()

#2. Preproccessing

In [38]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
# Lemmatization
lemmatizer = WordNetLemmatizer()

In [44]:
f = f.replace(r'\n', ' ')
f = f.lower()
tokens = pd.Series(word_tokenize(f))
tokens = tokens.loc[~tokens.isin(stopwords.words('english'))]
tokens = tokens.loc[~tokens.isin(list(string.punctuation + string.digits + "'" + '"' + '”' + '“' + '’' + '‘'))]
lemmatizer = nltk.WordNetLemmatizer()
tokens = tokens.apply(lambda word: lemmatizer.lemmatize(word))

tokens

0             alice
3         adventure
5        wonderland
7             lewis
8           carroll
            ...    
34622    child-life
34626         happy
34627        summer
34628           day
34631           end
Length: 12461, dtype: object

In [39]:
# Split text into tokens
tokens = pd.Series(word_tokenize(f))
# deleting stopwords
stop_words = stopwords.words("english")
tokens = tokens.loc[~tokens.isin(stop_words)]
tokens = tokens.loc[~tokens.isin(list(string.punctuation + string.digits + "'" + '"' + '”' + '“' + '’' + '‘'))]

In [5]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
# Lemmatization
lemmatizer = WordNetLemmatizer()
tokens = tokens.apply(lambda w: lemmatizer.lemmatize(w))


In [48]:
#grouping tokens in whole text into chapters and converting it to Dataframe
chapters = []

chapter_nmb = 0
for word in tokens.iteritems():
  if (word[1] == 'chapter'):
    if chapter_nmb == 12:
      chapter_nmb = 1
    else:
      chapter_nmb += 1
  chapters.append({ 'chapter': chapter_nmb, 'word': word[1] })

chapter_text = pd.DataFrame(chapters)

In [49]:
chapter_text["chapter"].value_counts()

4     1208
6     1205
8     1162
9     1114
7     1098
10    1016
5     1009
12     988
2      976
1      970
11     884
3      821
0       10
Name: chapter, dtype: int64

#3. Here we'r finding top-10 words in every chapter

In [50]:
def topWords(df, chapter):
  current_chapter = df[df.chapter == chapter]
  unique_words = current_chapter[current_chapter.word != 'alice'].word.unique()

  words_count = []

  for w in unique_words:
    words_count.append({
      'word': w,
      'count': current_chapter.loc[current_chapter.word == w].word.count()
    })
  
  counted = pd.DataFrame(words_count)
  return counted.sort_values(by=['count'], ascending=False).head(10)

for i in range(12):
  i +=1
  print('Top 10 words in chapter ' + str(i))
  print(topWords(chapter_text, i))
  print('----------------------')

Top 10 words in chapter 1
       word  count
225  little     15
93     like     11
50      way     11
84      see     10
48    think      9
296    door      9
126     one      8
61     time      8
22    could      8
159    said      8
----------------------
Top 10 words in chapter 2
       word  count
29   little     16
366   mouse     16
107    said     12
34     dear     11
52       go     11
171   thing     10
20     foot     10
15     like      9
2      pool      9
27       oh      9
----------------------
Top 10 words in chapter 3
      word  count
55    said     34
56   mouse     21
169   dodo     12
45    know     11
201    one      8
3     long      7
65    soon      7
10    bird      6
37    lory      6
25     dry      6
----------------------
Top 10 words in chapter 4
        word  count
4     little     24
2     rabbit     16
77      said     14
198      one     14
5       bill     13
23       get      9
15     heard      9
25      sure      9
224  thought      9
168    quit

#4. Here we are finding top-10 verbs in sent with Alce. We should split text not into chapters, but by symbols '.'

In [51]:
f = f.replace('”', '')
f = f.replace('?', '')
f = f.replace('!', '')
sentences =  f.split('.')
sentences_tokenized = [word_tokenize(sent) for sent in sentences]
len(sentences_tokenized)

1001

In [52]:
alice_list = []
k = 0
for sent in sentences_tokenized:
  k = 0
  for word in sent:
    if word =='alice':
      k +=1
  if k>0:
    alice_list.append(sent)

len(alice_list)

366

In [53]:
filter = []
for sent in alice_list:
  sent = [token for token in sent if token not in stop_words]
  sent = [lemmatizer.lemmatize(token, pos = 'v') for token in sent]
  postag = nltk.pos_tag(sent)
  for w in postag:
    if (w[1] == 'VB') or (w[1] == 'VBP'):
      filter.append(w[0])

In [54]:
print(filter)

['get', 'peep', 'think', 'consider', 'make', 'make', 'worth', 'get', 'run', 'think', 'say', 'think', 'occur', 'wonder', 'seem', 'see', 'burn', 'run', 'see', 'go', 'consider', 'go', 'like', 'think', 'stop', 'find', 'think', 'think', 'brave', 'think', 'say', 'let', 'see', 'say', 'wonder', 'think', 'say', 'begin', 'wonder', 'begin', 'get', 'go', 'say', 'see', 'go', 'say', 'turn', 'find', 'round', 'try', 'walk', 'wonder', 'get', 'make', 'belong', 'open', 'come', 'try', 'find', 'kneel', 'get', 'wander', 'get', 'go', 'think', 'little', 'see', 'begin', 'think', 'go', 'find', 'find', 'say', 'say', 'wise', 'go', 'find', 'say', 'shut', 'wait', 'see', 'end', 'know', 'say', 'go', 'find', 'decide', 'go', 'get', 'find', 'go', 'find', 'reach', 'see', 'try', 'try', 'use', 'advise', 'leave', 'give', 'follow', 'bring', 'remember', 'try', 'cheat', 'pretend', 'think', 'pretend', '’', 'leave', 'find', 'say', 'make', 'reach', 'make', 'get', 'happen', 'say', 'feel', 'find', 'happen', 'get', 'expect', 'seem',

In [55]:
top_verbs = []
for w in filter:
  counter = 1
  for v in filter:
    if w == v:
      counter +=1
      filter.remove(v)
  top_verbs.append((w, counter))

In [56]:
sorted(top_verbs, key=lambda v: -v[1])[:10]

[('say', 189),
 ('go', 85),
 ('think', 70),
 ('get', 48),
 ('know', 36),
 ('see', 34),
 ('make', 33),
 ('say', 33),
 ('find', 27),
 ('begin', 26)]